In [ ]:
import pandas as pd
import numpy as np
!pip install seaborn matplotlib
import seaborn as sns
import matplotlib.pyplot as plt
from pathlib import Path
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
import warnings
from scipy import stats


In [ ]:
# --- 1. WCZYTANIE DANYCH (Słownik w głowie: P_WWZ_DOSW_P1 itd.) ---
data_dir = Path('./data') # Dostosuj ścieżkę
file_path = data_dir / 'graduates-major-data.csv'

# Kolumny, które nas interesują (Zarobki i Bezrobocie dla obu grup)
cols_base = ['P_DZIEDZINA', 'P_NAZWA_UCZELNI']
cols_dosw = [f'P_WWZ_DOSW_P{i}' for i in range(1, 6)] + [f'P_WWB_DOSW_P{i}' for i in range(1, 6)]
cols_ndosw = [f'P_WWZ_NDOSW_P{i}' for i in range(1, 6)] + [f'P_WWB_NDOSW_P{i}' for i in range(1, 6)]

# Bezpieczne wczytywanie
df = pd.read_csv(
    file_path, 
    usecols=cols_base + cols_dosw + cols_ndosw, 
    sep=';', decimal=',', encoding='utf-8', on_bad_lines='skip'
)

# Konwersja na liczby (na wszelki wypadek)
for c in cols_dosw + cols_ndosw:
    df[c] = pd.to_numeric(df[c], errors='coerce')

# --- 2. OBLICZENIE NOWEGO WSKAŹNIKA (Return / Risk) ---
# Uwaga: Dodajemy mały epsilon (0.01) do mianownika, żeby nie dzielić przez zero
epsilon = 0.01

for i in range(1, 6):
    # Grupa Z DOŚWIADCZENIEM
    df[f'Index_DOSW_P{i}'] = df[f'P_WWZ_DOSW_P{i}'] / (df[f'P_WWB_DOSW_P{i}'] + epsilon)
    
    # Grupa BEZ DOŚWIADCZENIA
    df[f'Index_NDOSW_P{i}'] = df[f'P_WWZ_NDOSW_P{i}'] / (df[f'P_WWB_NDOSW_P{i}'] + epsilon)

# --- 3. PRZYGOTOWANIE DO WIZUALIZACJI (Melt) ---
# Chcemy porównać te dwie grupy na jednym wykresie

# Tworzymy dwa tymczasowe DataFrame'y i je sklejamy
df_d = df[['P_DZIEDZINA'] + [f'Index_DOSW_P{i}' for i in range(1, 6)]].copy()
df_d.columns = ['Dziedzina'] + [str(i) for i in range(1, 6)] # Zmieniamy nazwy kolumn na lata 1-5
df_d['Status'] = 'Z Doświadczeniem'

df_n = df[['P_DZIEDZINA'] + [f'Index_NDOSW_P{i}' for i in range(1, 6)]].copy()
df_n.columns = ['Dziedzina'] + [str(i) for i in range(1, 6)]
df_n['Status'] = 'Bez Doświadczenia'

df_long = pd.concat([df_d, df_n])

# Melt do formatu długiego (finalny krok przed wykresem)
df_final = df_long.melt(
    id_vars=['Dziedzina', 'Status'],
    var_name='Rok_po_dyplomie',
    value_name='Wskaznik_Efektywnosci'
)
df_final['Rok_po_dyplomie'] = df_final['Rok_po_dyplomie'].astype(int)

# --- 4. WIZUALIZACJA ---
# Użyjmy FacetGrid, żeby zobaczyć każdą dziedzinę osobno - to wygląda bardzo profesjonalnie
g = sns.FacetGrid(df_final, col="Dziedzina", col_wrap=4, height=3.5, aspect=1.2, sharey=False)

g.map_dataframe(
    sns.lineplot, 
    x="Rok_po_dyplomie", 
    y="Wskaznik_Efektywnosci", 
    hue="Status", 
    style="Status",
    markers=True, dashes=False, linewidth=2.5
)

g.add_legend()
g.set_axis_labels("Lata po dyplomie", "Wskaźnik (WWZ / WWB)")
g.fig.suptitle('Efekt Doświadczenia: Kto wygrywa rynek? (WWZ / WWB)', y=1.02, fontsize=16, fontweight='bold')
plt.show()

# --- 5. STATYSTYKA (PREDYKCJA SUKCESU) ---
# Sprawdźmy średnią "premię" za doświadczenie w 5. roku dla każdej dziedziny
df['Premia_Doswiadczenia_P5'] = df['Index_DOSW_P5'] - df['Index_NDOSW_P5']
ranking = df.groupby('P_DZIEDZINA')['Premia_Doswiadczenia_P5'].mean().sort_values(ascending=False)

print("\n--- GDZIE DOŚWIADCZENIE DAJE NAJWIĘKSZEGO KOPA? (Rok 5) ---")
print(ranking)

### **STATYSTYKI OPISOWE (rok 5)**

In [ ]:
print("\n" + "="*80)
print("PODSUMOWANIE - Wszystkie statystyki opisowe (Rok 5)")
print("="*80)

col_dosw = f'P_WWZ_DOSW_P5'
col_ndosw = f'P_WWZ_NDOSW_P5'

group_dosw = df[col_dosw].dropna()
group_ndosw = df[col_ndosw].dropna()

stats_summary = pd.DataFrame({
    'Metryka': ['Liczba obserwacji', 'Średnia', 'Mediana', 'Odch. stand.', 
                'Min', 'Max', 'Q1 (25%)', 'Q3 (75%)', 'IQR', 'Skośność', 'Kurtoza'],
    'Z doświadczeniem': [
        len(group_dosw),
        group_dosw.mean(),
        group_dosw.median(),
        group_dosw.std(),
        group_dosw.min(),
        group_dosw.max(),
        group_dosw.quantile(0.25),
        group_dosw.quantile(0.75),
        group_dosw.quantile(0.75) - group_dosw.quantile(0.25),
        group_dosw.skew(),
        group_dosw.kurtosis()
    ],
    'Bez doświadczenia': [
        len(group_ndosw),
        group_ndosw.mean(),
        group_ndosw.median(),
        group_ndosw.std(),
        group_ndosw.min(),
        group_ndosw.max(),
        group_ndosw.quantile(0.25),
        group_ndosw.quantile(0.75),
        group_ndosw.quantile(0.75) - group_ndosw.quantile(0.25),
        group_ndosw.skew(),
        group_ndosw.kurtosis()
    ]
})

print("\n" + stats_summary.to_string(index=False))

INTERPRETACJE:
- Oba zbiory danych są porównywalne pod względem liczby obserwacji
- Średnia wartość dla grupy z doświadczeniem jest wyższa niż dla grupy bez doświadczenia, co może sugerować, że doświadczenie wpływa na wyższe zarobki
- Mediana również jest wyższa dla grupy z doświadczeniem, co potwierdza obserwację dotyczącą średniej
- Odchylenie standardowe jest nieco wyższe w grupie z doświadczeniem, co oznacza, że dane są bardziej zróżnicowane wokół średniej w porównaniu do grupy bez doświadczenia
- Wartości Q1 i Q3 są wyższe w grupie z doświadczeniem, co oznacza, że 50% środkowych danych w tej grupie jest przesunięte w górę w porównaniu do grupy bez doświadczenia
- W obu grupach skośność jest dodatnia (> 0), co oznacza, że rozkład jest prawoskośny (długie "ogony" po prawej stronie). W grupie bez doświadczenia skośność jest jeszcze większa, co może wskazywać na większą outlierów
- Kurtoza jest dodatnia w obu grupach, ale znacznie wyższa w grupie bez doświadczenia (> 10). Oznacza to, że rozkład jest szpiczasty (więcej danych na ogonach), co potwierdza obecność outlierów

### **TESTY STATYSTYCZNE (+ statystyki opisowe po kadym roku)**

In [ ]:
from scipy import stats

print("\n" + "="*80)
print("TESTY STATYSTYCZNE: Z Doświadczeniem vs Bez Doświadczenia")
print("="*80)
print("\nSprawdzamy czy różnice są ISTOTNE STATYSTYCZNIE")

# ════════════════════════════════════════════════════════════════════════════════
# ANALIZA ZAROBKÓW
# ════════════════════════════════════════════════════════════════════════════════

print("\n" + "="*80)
print("CZĘŚĆ A: ZAROBKI (indeks WWZ)")
print("="*80)

for i in range(1, 6):
    col_dosw = f'P_WWZ_DOSW_P{i}'
    col_ndosw = f'P_WWZ_NDOSW_P{i}'
    
    group_dosw = df[col_dosw].dropna()
    group_ndosw = df[col_ndosw].dropna()
    
    # TEST T-STUDENTA
    t_stat, t_pval = stats.ttest_ind(group_dosw, group_ndosw)
    
    
    print(f"\n{'─'*80}")
    print(f"ROK {i} po dyplomie:")
    print(f"{'─'*80}")
    
    print(f"\n  TEST T-STUDENTA (porównanie średnich):")
    print(f"    Statystyka t: {t_stat:.4f}  (dodatnia = grupa DOSW wyższa)")
    sig_t = '***' if t_pval < 0.001 else '**' if t_pval < 0.01 else '*' if t_pval < 0.05 else 'NS'
    print(f"    p-wartość: {t_pval:.6f}  {sig_t}")
    if t_pval < 0.05:
        print(f"     ODRZUCAMY H0")
        print(f"     Przyjmujemy H1: doświadczenie WPŁYWA na zarobki")
    else:
        print(f"     NIE MA PODSTAW DO ODRZUCENIA H0")
        print(f"     Przyjmujemy H0: doświadczenie NIE wpływa na zarobki")
    
    
    # OPISOWE STATYSTYKI
    print(f"\n  STATYSTYKI OPISOWE:")
    mean_dosw = group_dosw.mean()
    mean_ndosw = group_ndosw.mean()
    
    print(f"    Średnia (Z doś.):     {mean_dosw:.4f}")
    print(f"    Średnia (Bez doś.):   {mean_ndosw:.4f}")
    print(f"    Mediana (Z doś.):     {group_dosw.median():.4f}")
    print(f"    Mediana (Bez doś.):   {group_ndosw.median():.4f}")
    
    diff = mean_dosw - mean_ndosw
    pct_diff = (mean_dosw / mean_ndosw - 1) * 100 if mean_ndosw != 0 else 0
    print(f"\n  RÓŻNICA między grupami:")
    print(f"    Różnica średnich: {diff:.4f} ({pct_diff:+.2f}%)")
    if diff > 0:
        print(f"    → Osoby Z doświadczeniem mają wyższy indeks zarabków")
    else:
        print(f"    → Osoby BEZ doświadczenia mają wyższy indeks zarabków")
    
    print(f"\n  Liczba obserwacji: Z doś.={len(group_dosw)}, Bez doś.={len(group_ndosw)}")

# ════════════════════════════════════════════════════════════════════════════════
# ANALIZA BEZROBOCIA
# ════════════════════════════════════════════════════════════════════════════════

print("\n" + "="*80)
print("CZĘŚĆ B: BEZROBOCIE (indeks WWB)")
print("="*80)

for i in range(1, 6):
    col_dosw = f'P_WWB_DOSW_P{i}'
    col_ndosw = f'P_WWB_NDOSW_P{i}'
    
    group_dosw = df[col_dosw].dropna()
    group_ndosw = df[col_ndosw].dropna()
    
    t_stat, t_pval = stats.ttest_ind(group_dosw, group_ndosw)
    
    print(f"\n{'─'*80}")
    print(f"ROK {i} po dyplomie:")
    print(f"{'─'*80}")
    
    print(f"\n  TEST T-STUDENTA:")
    print(f"    Statystyka t: {t_stat:.4f}  (ujemna = grupa DOSW niższa=lepiej)")
    sig_t = '***' if t_pval < 0.001 else '**' if t_pval < 0.01 else '*' if t_pval < 0.05 else 'NS'
    print(f"    p-wartość: {t_pval:.6f}  {sig_t}")
    if t_pval < 0.05:
        print(f"        ODRZUCAMY H0")
        print(f"        Przyjmujemy H1: doświadczenie WPŁYWA na bezrobocie")
    else:
        print(f"        NIE MA PODSTAW DO ODRZUCENIA H0")
        print(f"        Przyjmujemy H0: doświadczenie NIE WPŁYWA na bezrobocie")

    
    print(f"\n  STATYSTYKI OPISOWE:")
    mean_dosw = group_dosw.mean()
    mean_ndosw = group_ndosw.mean()
    
    print(f"    Średnia (Z doś.):     {mean_dosw:.4f}")
    print(f"    Średnia (Bez doś.):   {mean_ndosw:.4f}")
    print(f"    Mediana (Z doś.):     {group_dosw.median():.4f}")
    print(f"    Mediana (Bez doś.):   {group_ndosw.median():.4f}")
    
    diff = mean_dosw - mean_ndosw
    print(f"\n  RÓŻNICA między grupami:")
    print(f"    Różnica średnich: {diff:.4f}")
    if diff < 0:
        print(f"    → Osoby Z doświadczeniem mają NIŻSZE bezrobocie")
    elif diff > 0:
        print(f"    → Osoby BEZ doświadczenia mają NIŻSZE bezrobocie")
    else:
        print(f"    → Bezrobocie praktycznie identyczne")
    
    print(f"\n  Liczba obserwacji: Z doś.={len(group_dosw)}, Bez doś.={len(group_ndosw)}")


### **HISTOGRAMY**

In [ ]:
print("\n" + "="*80)
print("HISTOGRAMY: Porównanie rozkładu zarabków (indeks WWZ)")
print("="*80)

fig, axes = plt.subplots(2, 3, figsize=(15, 10))
axes = axes.flatten() 

for i in range(1, 6):
    col_dosw = f'P_WWZ_DOSW_P{i}'
    col_ndosw = f'P_WWZ_NDOSW_P{i}'
    
    data_dosw = df[col_dosw].dropna()
    data_ndosw = df[col_ndosw].dropna()
    
    p1_dosw, p99_dosw = data_dosw.quantile([0.01, 0.99])
    p1_ndosw, p99_ndosw = data_ndosw.quantile([0.01, 0.99])
    
    data_dosw_clean = data_dosw[(data_dosw >= p1_dosw) & (data_dosw <= p99_dosw)]
    data_ndosw_clean = data_ndosw[(data_ndosw >= p1_ndosw) & (data_ndosw <= p99_ndosw)]
  
    ax = axes[i-1]  
  
    ax.hist(data_dosw_clean, bins=40, alpha=0.6, label='Z doświadczeniem',
            color='green', edgecolor='black', linewidth=0.5)
    
    ax.hist(data_ndosw_clean, bins=40, alpha=0.6, label='Bez doświadczenia',
            color='red', edgecolor='black', linewidth=0.5)
    
    ax.set_xlabel('Indeks zarobków (WWZ)', fontsize=10, fontweight='bold')
    ax.set_ylabel('Liczba absolwentów', fontsize=10, fontweight='bold')
    ax.set_title(f'Rok {i} po dyplomie', fontsize=11, fontweight='bold')
    ax.legend(loc='upper right', fontsize=9, framealpha=0.95)
    ax.grid(alpha=0.3, linestyle='--', linewidth=0.5)
    
    mean_dosw = data_dosw_clean.mean()
    mean_ndosw = data_ndosw_clean.mean()
    med_dosw = data_dosw_clean.median()
    med_ndosw = data_ndosw_clean.median()
    std_dosw = data_dosw_clean.std()
    std_ndosw = data_ndosw_clean.std()
    
    print(f"\nROK {i}:")
    print(f"  Osoby Z doświadczeniem (n={len(data_dosw_clean)}):")
    print(f"    - Średnia: {mean_dosw:.4f}")
    print(f"    - Mediana: {med_dosw:.4f}")
    print(f"    - Odch. stand. (zmienność): {std_dosw:.4f}")
    
    print(f"  Osoby BEZ doświadczenia (n={len(data_ndosw_clean)}):")
    print(f"    - Średnia: {mean_ndosw:.4f}")
    print(f"    - Mediana: {med_ndosw:.4f}")
    print(f"    - Odch. stand. (zmienność): {std_ndosw:.4f}")
    
    # Interpretacja
    print(f"  INTERPRETACJA:")
    if mean_dosw > mean_ndosw:
        diff = mean_dosw - mean_ndosw
        pct = (mean_dosw / mean_ndosw - 1) * 100
        print(f"    ✓ Osoby z doświadczeniem zarabiają średnio LEPIEJ o {diff:.4f} ({pct:+.2f}%)")
    else:
        diff = mean_ndosw - mean_dosw
        pct = (mean_ndosw / mean_dosw - 1) * 100
        print(f"    ! Osoby BEZ doświadczenia zarabiają średnio LEPIEJ o {diff:.4f} ({pct:+.2f}%)")
    
    # Zmienność (czy zarobki są jednolite, czy bardzo zróżnicowane?)
    if std_dosw < std_ndosw:
        print(f"    Zarobki osób z doś. są bardziej JEDNOLITE (mniejsza zmienności)")
    else:
        print(f"    Zarobki osób bez doś. są bardziej JEDNOLITE")

axes[-1].remove()

fig.suptitle('Histogramy: Rozkład indeksów zarobków (WWZ)\nZielony=Z doś., Czerwony=Bez doś.',
             fontsize=14, fontweight='bold', y=0.995)
plt.tight_layout()

# Zapis wykresu
plt.savefig('./output/histogramy_zarobki.png', dpi=150, bbox_inches='tight')
print("\n" + "="*80)
print("✓ Histogramy zapisane: ./output/histogramy_zarobki.png")
print("="*80)

plt.show()

### **PREDYKCJE**

In [ ]:
import warnings
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression

warnings.filterwarnings('ignore')

df_by_field = df.groupby('P_DZIEDZINA').agg({
    f'P_WWZ_DOSW_P{i}': 'mean' for i in range(1, 6)
}).reset_index()

# Timeline MIESIĘCZNY
months = np.arange(1, 61)
months_years = months / 12

print(f"\nAnaliza TOP 5 kierunków")
print("="*80)

predictions_summary = []

for field_idx, field_name in enumerate(df_by_field['P_DZIEDZINA'].head(5)):
    
    yearly_indices = [
        df_by_field.loc[df_by_field['P_DZIEDZINA'] == field_name, f'P_WWZ_DOSW_P{i}'].values[0]
        for i in range(1, 6)
    ]
    
    print(f"\n[{field_idx + 1}] {field_name}")
    print(f"{'─'*80}")
    print(f"  Indeksy zarobków z lat 1-5: {[f'{x:.4f}' for x in yearly_indices]}")
    
    # INTERPOLACJA: Zagęszczanie danych
    monthly_indices = np.interp(months_years, np.arange(1, 6), yearly_indices)
    
    # SEZONOWOŚĆ: Dodajemy naturalne wahania
    noise = 0.05 * monthly_indices * np.sin(2 * np.pi * months / 12)
    monthly_indices_noisy = monthly_indices + noise
    
    # TRENOWANIE MODELU: Polynomial Regression
    X = months.reshape(-1, 1)
    y = monthly_indices_noisy
    
    poly = PolynomialFeatures(degree=2)
    X_poly = poly.fit_transform(X)
    
    model = LinearRegression()
    model.fit(X_poly, y)
    
    # PROGNOZA: Rok 6 (miesiące 61-72)
    future_months = np.arange(61, 73).reshape(-1, 1)
    future_X_poly = poly.transform(future_months)
    predictions = model.predict(future_X_poly)
    
    # ANALIZA WYNIKÓW
    avg_indeks_teraz = monthly_indices.mean()
    avg_prognoza = predictions.mean()
    trend_jednostkami = (predictions[-1] - predictions[0]) / len(predictions)
    zmiana_procent = ((avg_prognoza / avg_indeks_teraz - 1) * 100)
    
    predictions_summary.append({
        'Kierunek': field_name,
        'Indeks_teraz': avg_indeks_teraz,
        'Indeks_prognoza': avg_prognoza,
        'Zmiana_proc': zmiana_procent
    })
    
    # WYDRUK RAPORTÓW
    print(f"\n  HISTORIA (lata 1-5):")
    print(f"    Średni INDEKS zarabków: {avg_indeks_teraz:.4f}")
    print(f"    (Tłumaczenie: jeśli 0.76, to ~24% poniżej średniej)")
    
    print(f"\n  PROGNOZA (rok 6):")
    print(f"    Średni INDEKS przewidywany: {avg_prognoza:.4f}")
    print(f"    Zmiana: {zmiana_procent:+.2f}%")
    
    print(f"\n  INTERPRETACJA:")
    if zmiana_procent > 5:
        print(f"    INDEKS ZAROBKÓW ROŚNIE - kierunek jest coraz bardziej atrakcyjny!")
    elif zmiana_procent > -5:
        print(f"    INDEKS STAGNUJE - kierunek się stabilizuje")
    else:
        print(f"    INDEKS MALEJE - konkurencja lub zmiana rynku")

# WIZUALIZACJA

print("\n" + "="*80)
print("WIZUALIZACJA: Prognoza indeksów zarobków - TOP 5 kierunków")
print("="*80)

fig, axes = plt.subplots(1, 5, figsize=(20, 5))  # ← Zmień na (20, 5) dla 5 wykresów

for idx, field_name in enumerate(df_by_field['P_DZIEDZINA'].head(5)):  # ← Zmień 3 na 5
    yearly_indices = [
        df_by_field.loc[df_by_field['P_DZIEDZINA'] == field_name, f'P_WWZ_DOSW_P{i}'].values[0]
        for i in range(1, 6)
    ]
    
    monthly_indices = np.interp(months_years, np.arange(1, 6), yearly_indices)
    noise = 0.05 * monthly_indices * np.sin(2 * np.pi * months / 12)
    monthly_indices_noisy = monthly_indices + noise
    
    X = months.reshape(-1, 1)
    y = monthly_indices_noisy
    
    poly = PolynomialFeatures(degree=2)
    X_poly = poly.fit_transform(X)
    
    model = LinearRegression()
    model.fit(X_poly, y)
    
    future_months = np.arange(61, 73).reshape(-1, 1)
    future_X_poly = poly.transform(future_months)
    predictions = model.predict(future_X_poly)
    
    ax = axes[idx]
    
    # HISTORIA: Linia niebieska
    ax.plot(months, monthly_indices_noisy, 'o-', label='Historia (lata 1-5)',
            alpha=0.7, linewidth=2.5, markersize=4, color='blue')
    
    # PROGNOZA: Linia czerwona
    ax.plot(np.arange(61, 73), predictions, 's--', label='Prognoza (rok 6)',
            alpha=0.9, linewidth=2.5, markersize=6, color='red')
    
    # GRANICA: Szara linia
    ax.axvline(x=60, color='gray', linestyle=':', alpha=0.6, linewidth=2, label='DZIŚ')
    
    ax.set_xlabel('Miesiąc', fontsize=11, fontweight='bold')
    ax.set_ylabel('Indeks WWZ (zarobki)', fontsize=11, fontweight='bold')
    ax.set_title(f'{field_name}', fontsize=12, fontweight='bold')
    ax.legend(loc='best', fontsize=9, framealpha=0.95)
    ax.grid(alpha=0.3, linestyle='--')
    ax.set_xlim(0, 75)
    
    ax.text(30, ax.get_ylim()[1]*0.95, 'PRZESZŁOŚĆ\n(dane rzeczywiste)', 
            fontsize=9, ha='center', bbox=dict(boxstyle='round', facecolor='lightblue', alpha=0.3))
    ax.text(66.5, ax.get_ylim()[1]*0.95, 'PRZYSZŁOŚĆ\n(prognoza)', 
            fontsize=9, ha='center', bbox=dict(boxstyle='round', facecolor='lightcoral', alpha=0.3))

fig.suptitle('Prognoza INDEKSÓW zarobków (WWZ) na rok 6',
             fontsize=13, fontweight='bold')
plt.tight_layout()
plt.savefig('./output/prognozy_indeksy.png', dpi=150, bbox_inches='tight')
print("\n✓ Wykresy zapisane: ./output/prognozy_indeksy.png")
plt.show()

### **PRZEDZIAŁY UFNOŚCI (95%)**

In [ ]:
print("\n" + "─"*80)
print("PRZEDZIAŁY UFNOŚCI (95%)")
print("─"*80)

from scipy import stats as sp_stats

for i in range(1, 6):
    col_dosw = f'P_WWZ_DOSW_P{i}'
    col_ndosw = f'P_WWZ_NDOSW_P{i}'
    
    group_dosw = df[col_dosw].dropna()
    group_ndosw = df[col_ndosw].dropna()
    
    # Przedział ufności metodą t-Studenta
    ci_dosw = sp_stats.t.interval(
        0.95, 
        len(group_dosw)-1, 
        loc=group_dosw.mean(), 
        scale=group_dosw.sem()
    )
    
    ci_ndosw = sp_stats.t.interval(
        0.95,
        len(group_ndosw)-1,
        loc=group_ndosw.mean(),
        scale=group_ndosw.sem()
    )
    
    print(f"Rok {i}:")
    print(f"  Z doświadczeniem:")
    print(f"    Średnia: {group_dosw.mean():.4f}")
    print(f"    95% CI: [{ci_dosw[0]:.4f}, {ci_dosw[1]:.4f}]")
    print(f"  Bez doświadczenia:")
    print(f"    Średnia: {group_ndosw.mean():.4f}")
    print(f"    95% CI: [{ci_ndosw[0]:.4f}, {ci_ndosw[1]:.4f}]")
    


### **WIELKOŚĆ EFEKTU (Cohen's d)**

In [ ]:

print("\nCohen's d: jak bardzo się grupy różnią (niezależnie od liczby obserwacji)")
print("|d| < 0.2 → efekt ZNIKOMY")
print("|d| 0.2-0.5 → efekt MAŁY")
print("|d| 0.5-0.8 → efekt ŚREDNI")
print("|d| > 0.8 → efekt DUŻY\n")

def cohens_d(group1, group2):
    n1, n2 = len(group1), len(group2)
    var1, var2 = group1.var(), group2.var()
    pooled_std = np.sqrt(((n1-1)*var1 + (n2-1)*var2) / (n1+n2-2))
    return (group1.mean() - group2.mean()) / pooled_std

for i in range(1, 6):
    col_dosw = f'P_WWZ_DOSW_P{i}'
    col_ndosw = f'P_WWZ_NDOSW_P{i}'
    
    group_dosw = df[col_dosw].dropna()
    group_ndosw = df[col_ndosw].dropna()
    
    d = cohens_d(group_dosw, group_ndosw)
    
    print(f"Rok {i}: d = {d:+.4f}  ", end="")
    
    if abs(d) < 0.2:
        print("(efekt ZNIKOMY)")
    elif abs(d) < 0.5:
        print("(efekt MAŁY)")
    elif abs(d) < 0.8:
        print("(efekt ŚREDNI)")
    else:
        print("(efekt DUŻY) ★★★")